# The Most in Demand Skills for Data Scientists
## What are employers looking for?
## By Jeff Hale

I'm going to explore which skills and software for Data Scientists are in demand in the US in October 2018 by looking at online job listing sites. I'll be using plotly and pandas to make interactive visualizations for this project.

### See [this Medium article](https://towardsdatascience.com/the-most-in-demand-skills-for-data-scientists-4a4a8db896db) for discussion.


This notebook includes additional analyses not in the Medium article and presents the findings in a slightly different order.

# Please upvote this Kaggle kernel if you find it helpful.

In [ ]:
# import the usual frameworks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings

from IPython.core.display import display, HTML
from sklearn.preprocessing import MinMaxScaler
    
# import plotly 
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls

# for color scales in plotly
import colorlover as cl 

# configure things
warnings.filterwarnings('ignore')

pd.options.display.float_format = '{:,.2f}'.format  
pd.options.display.max_columns = 999

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2
%matplotlib inline

List package versions for reproducibility.

In [ ]:
#!pip list
# !pwd
# !ls

# Part 1: Data Science Software

Read in the data from the csv. The Google sheet that holds the data is available [here](https://docs.google.com/spreadsheets/d/1lac1H2IgCDCs9LLTQL6yb6MUPN1u4C5fJv_6YjipIaM/edit?usp=sharing).

In [ ]:
df = pd.read_csv(
    '../input/ds-job-listing-technology/ds_job_listing_software.csv',
    usecols=['Keyword','LinkedIn', 'Indeed', 'SimplyHired', 'Monster'],
    skiprows=0,
    nrows=37, 
    thousands=',',
    index_col=0,
   
)
df

Cool. We used the read_csv parameters to give us just what we wanted.

## Basic Data Exploration
Let's see what the data look like.

In [ ]:
df.info()

In [ ]:
df.describe()

# Scale and Aggregate 
Scale each column. For each column we'll use MinMaxScaler to subtract the minumum and divide by the original max - original min.

In [ ]:
scale = MinMaxScaler()
scaled_df = pd.DataFrame(
    scale.fit_transform(df), 
    columns = df.columns,
    index = df.index)    

In [ ]:
scaled_df.head()

### Scaled Online Job Listings

Let's make a combined score for the large sites by taking the mean of the LinkedIn, Indeed, Monster, and SimplyHired scores.

In [ ]:
scaled_df['combined'] = scaled_df[["LinkedIn", "Indeed", "SimplyHired", "Monster"]].mean(axis = 1)
scaled_df.head()

# Charts with Plotly
Let's make interactive charts with plotly. 

## Let's look at the top 10 by site
Based on the top 10 on LinkedIn, the largest site.

In [ ]:
num_sites = 8

y_linkedin = scaled_df.iloc[:num_sites, 0]
y_indeed = scaled_df.iloc[:num_sites, 1]
y_monster = scaled_df.iloc[:num_sites, 2]
y_simply = scaled_df.iloc[:num_sites, 3]

y_linkedin

In [ ]:
indeed = go.Bar(x=scaled_df.index, y=y_indeed, name = "Indeed")
simply = go.Bar(x=scaled_df.index, y=y_simply, name="SimplyHired")
monster = go.Bar(x=scaled_df.index, y=y_monster, name="Monster")
linked = go.Bar(x=scaled_df.index, y=y_linkedin, name="LinkedIn")

data = [linked, indeed, simply, monster]

layout = go.Layout(
    barmode='group',
    title="Top Software in Job Listings for Data Scientists",
    xaxis={'title': 'Software'},
    yaxis={'title': "Scaled Listings", 'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Let's clean things up for publication

In [ ]:
p_s_df = scaled_df * 100
p_s_df = p_s_df.round(2)
p_s_df.columns = ['LinkedIn', 'Indeed', 'SimplyHired', 'Monster', 'Score']
p_s_df = p_s_df.sort_values(by=['Score'], ascending = False)
p_s_df.head()

Let's rename Microsoft Office. Otherwise it gets cut off.

In [ ]:
p_s_df.rename(index = {'Microsoft Office': 'MS Office'}, inplace = True)
p_s_df

## Let's look at the top 20 scores

In [ ]:
p_s_df_20 = p_s_df.iloc[:20,:]
p_s_df_20

## Let's make a bar chart of the top 20 combined scores

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 20)

data = [
    go.Bar(
        x=p_s_df_20.index,          
        y=p_s_df_20['Score'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = {
    'title': 'Top 20 Technology Skills in Data Scientist Job Listings',
    #'xaxis': {'title': 'Technology'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Let's make a bar chart of all the combined scores

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 44)

data = [
    go.Bar(
        x=p_s_df.index,          
        y=p_s_df['Score'],
        marker=dict(
            colorscale='Jet',
            #cauto=True,
            color=color_s,
        ),
        # text=p_s_df['Score'],
        # textposition='outside',
        # textfont=dict(size=10)
    )
]

layout = {
    'title': 'Technology Skills in Data Scientist Job Listings',
    #'xaxis': {'tickmode': 'linear'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## % of job listings with software skills


Total "data scientist" listings.

In [ ]:
total_ds = {
    'LinkedIn': 8610,
    'Indeed': 5138,
    'SimplyHired': 3829,
    'Monster': 3746,
}

In [ ]:
for key, value in total_ds.items():
    df[key + " %"] = df[key] / value * 100
df

Let's make an average of those % columns.

In [ ]:
df['avg_percent'] = df.iloc[:,-4:].mean(axis=1)
df

In [ ]:
df = df.sort_values(by="avg_percent", ascending = False)
df

## Let's chart the full list and then the top 20.

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 44)

data = [
    go.Bar(
        x=df.index,          
        y=df['avg_percent'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = dict(
    title="Technology Skills in Data Scientist Job Listings",
    yaxis=dict(title="% of Listings",)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 20)

data = [
    go.Bar(
        x=df.index,          
        y=df.iloc[:20, -1],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = dict(
    title="Top 20 Technology Skills in Data Scientist Job Listings",
    yaxis=dict(title="% of Listings",)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Glassdoor comparision

Glassdoor has some similar analysis from early 2017 [here](https://www.glassdoor.com/research/data-scientist-personas/). See [my larger Medium article](https://towardsdatascience.com/the-most-in-demand-skills-for-data-scientists-4a4a8db896db) for discussion.

In [ ]:
# percentage of data scientist jobs on glassdoor early 2017
glassdoor = {
    'Python': 72,
    'R': 64,
    'SQL': 51,
    'Spark': 27,
    'Hadoop': 39,
    'Java': 33,
    'SAS': 30,
    'Tableau': 14,
    'Hive': 17,
    'Matlab': 20
}

In [ ]:
# make a data frame of just these
# could just merge the series and df 

series_gd = pd.Series(glassdoor)
series_gd

In [ ]:
df_gd = pd.merge(
    df, 
    pd.DataFrame(
        series_gd, 
        columns=['gd_percent']), 
    right_index=True, 
    left_index=True, 
    how="inner"
)

# df_gd = df_gd[""]
# df_gd.columns = 
df_gd

In [ ]:
avg = go.Bar(x=df_gd.index, y=df_gd['avg_percent'], name="Oct. 2018 Averaged")
gd = go.Bar(x=df_gd.index, y=df_gd['gd_percent'], name="Early 2017 Glassdoor")


data = [gd, avg]

layout = go.Layout(
    barmode='group',
    title="Technology Skills 2017 Glassdoor Compared to 2018 Average",
    xaxis={'title': 'Technology'},
    yaxis={'title': '% of Listings'},
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# Part 2: General Data Science Skills
Let's look at general data science skills next.

In [ ]:
df_skills = pd.read_csv(
    '../input/data-scientist-general-skills-2018-revised/ds_general_skills_revised.csv',
    nrows=15,
    thousands=',',
    index_col=0,  
    )
df_skills

# this dataset was updated with "data engineering included 10/15/18"

All terms were searched with *"data scientist"* again. The skills are tricky because there is some overlap. 

I combined *AI* and *artificial intelligence* because one is the abbreviation of the other. I did the same for *NLP* and *natural language processing*. I combined these terms be adding the totals together and subtracting the number of listings that had both terms in them.

Let's rename the composite indices.

In [ ]:
df_skills.rename(index={'AI composite': 'AI', 'NLP composite': 'NLP'}, inplace = True)
df_skills

# Scale and Aggregate 
Scale each column. For each column we'll use MinMaxScaler to subtract the minumum and divide by the original max - original min.

In [ ]:
scale = MinMaxScaler()
scaled_df = pd.DataFrame(
    scale.fit_transform(df_skills), 
    columns = df_skills.columns,
    index = df_skills.index)    

In [ ]:
scaled_df

### Scaled Online Job Listings

Let's make a combined score for the large sites by taking the mean of the LinkedIn, Indeed, and SimplyHired scores.

In [ ]:
scaled_df['big_sites'] = scaled_df[["LinkedIn", "Indeed", "SimplyHired", "Monster"]].mean(axis = 1)
scaled_df.sort_values(by = 'big_sites', ascending = False)

## Let's look at them by site


In [ ]:
y_linkedin = scaled_df.iloc[:, 0]
y_indeed = scaled_df.iloc[:, 1]
y_simply = scaled_df.iloc[:, 2]
y_monster = scaled_df.iloc[:, 3]

y_linkedin

In [ ]:
indeed = go.Bar(x=scaled_df.index, y=y_indeed, name = "Indeed")
simply = go.Bar(x=scaled_df.index, y=y_simply, name="SimplyHired")
linked = go.Bar(x=scaled_df.index, y=y_linkedin, name="LinkedIn")
monster = go.Bar(x=scaled_df.index, y=y_monster, name="Monster")

data = [linked, indeed, simply, monster]

layout = go.Layout(
    barmode='group',
    title="Data Science Skills in Online Job Listings",
    xaxis={'title': 'Skills'},
    yaxis={'title': "Scaled Listings", 'separatethousands': True,
    }
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

Let's clean things up for publication

In [ ]:
p_s_df = scaled_df * 100
p_s_df = p_s_df.round(2)
p_s_df.columns = ['LinkedIn', 'Indeed', 'SimplyHired', 'Monster', 'Score']
p_s_df = p_s_df.sort_values(by=['Score'], ascending = False)
p_s_df

## Let's make a bar chart of the combined scores

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 14)

data = [
    go.Bar(
        x=p_s_df.index,          
        y=p_s_df['Score'],
        marker=dict(
            colorscale='Jet',
            #cauto=True,
            color=color_s,
        ),
        # text=p_s_df['Score'],
        # textposition='outside',
        # textfont=dict(size=10)
    )
]

layout = {
    'title': 'General Skills in Data Scientist Job Listings',
    'xaxis': {'tickmode': 'linear'},
    'yaxis': {'title': "Score"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

## Let's look at these general skills by % of occurrence, too.

In [ ]:
total_ds = {
    'LinkedIn': 8610,
    'Indeed': 5138,
    'SimplyHired': 3829,
    'Monster': 3746,
}

In [ ]:
for key, value in total_ds.items():
    df_skills[key + " %"] = df_skills[key] / value * 100
df_skills

Let's make an average of those % columns.

In [ ]:
df_skills['avg_percent'] = df_skills.iloc[:,-4:].mean(axis=1)
df_skills

In [ ]:
df_skills = df_skills.sort_values(by="avg_percent", ascending = False)
df_skills

## Let's chart the full list.

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 16)

data = [
    go.Bar(
        x=df_skills.index,          
        y=df_skills['avg_percent'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = dict(
    title="General Skills in Data Scientist Job Listings",
    yaxis=dict(title="Average % of Listings",)
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

# A few summary stats

In [ ]:
ds_results = pd.Series({
    'LinkedIn': 8610,
    'Indeed': 5138, 
    'SimplyHired': 3829,
    'Monster': 3746,
    'AngelList': 658
})
ds_results

In [ ]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 5)

data = [
    go.Bar(
        x=ds_results.index,          
        y=ds_results.values,
        marker=dict(
            colorscale='Jet',
            #cauto=True,
            color=color_s,
        ),
        # text=p_s_df['Score'],
        # textposition='outside',
        # textfont=dict(size=10)
    )
]

layout = {
    'title': "Data Scientist Job Listings",
    'xaxis': {'title': 'Website'},
    'yaxis': {'title': 'Listings', 'separatethousands': True,}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)



# Pleave upvote if you found this interesting or informative!

## Postscript: Configuring Plotly

Plotly is super cool, but setup can be a bit tricky and there is a lot of old documentation mixed in with relevant documentation. Here's how to get up and running.

Follow the instructions on [plotly's site](https://github.com/plotly/plotly.py).

To install on Mac/Linux from the command line with support for plotly's magic, depending on what you already have installed see the plotly docs. Here's what I had to run with a relatively new conda install:

conda install -c plotly plotly=3.3.0 <br>
conda install -c conda-forge nodejs <br>
conda install jupyterlab=0.34 "ipywidgets>=7.2" <br>
export NODE_OPTIONS=--max-old-space-size=4096 <br>
jupyter labextension install @jupyter-widgets/jupyterlab-manager@0.37 --no-build <br>
jupyter labextension install plotlywidget@0.4.0 --no-build <br>
jupyter labextension install @jupyterlab/plotly-extension@0.17.2 --no-build <br>
jupyter lab build <br>
unset NODE_OPTIONS <br>